In [207]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import re
import pickle

In [208]:
cornell_data = pickle.load(open('cornell_data.pkl','rb'))

In [209]:
cornell_data[1]['request'] + " " + cornell_data[1]['reply']

'not the hacking and gagging and spitting part please okay then how bout we try out some french cuisine saturday night'

In [210]:
#Create vocabulary 
vocab = {}
questions = []
answers = []
for pair in cornell_data:
    text = pair['request'] + " " + pair['reply']
    questions.append(pair['request'])
    answers.append(pair['reply'])
    for word in text.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [211]:
len(vocab)

44251

In [212]:
#Remove rare words
threshold = 10
c = 0
new_vocab = []
for word, count in vocab.items():
    if count >= threshold:
        c += 1
        new_vocab.append(word)

In [213]:
print("Total vocab: %d" % len(vocab))
print("Usable vocab: %d" % c)

Total vocab: 44251
Usable vocab: 8706


In [214]:
len(new_vocab)

8706

In [215]:
#Assign unique int value to each word in vocab
questions_vocab_to_int = {}
answers_vocab_to_int = {}
vocab_to_int = {}

word_num = 0

for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        answers_vocab_to_int[word] = word_num
        vocab_to_int[word] = word_num
        word_num += 1

In [216]:
#Add unique tokens to vocab
codes = ['<PAD>', '<EOS>', '<UNK>', '<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int) + 1
    answers_vocab_to_int[code] = len(answers_vocab_to_int) + 1
    vocab_to_int[code] = len(vocab_to_int) + 1
    new_vocab.append(code)

In [244]:
print(len(questions_vocab_to_int), len(answers_vocab_to_int))

8710 8710


In [217]:
#Converting from word->int to int->word
questions_int_to_vocab = {v_i: word for word,v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: word for word,v_i in answers_vocab_to_int.items()}
int_to_vocab = {v_i: word for word, v_i in vocab_to_int.items()}

In [218]:
#Add <EOS> token to end of answers
for i in range(len(cornell_data)):
    cornell_data[i]['reply'] += ' <EOS>'

In [219]:
#Replace words that are not in vocab with <UNK>

vocab_int = []

for pair in cornell_data:
    text = pair['request'] + ' ' + pair['reply']
    ints = []
    for word in text.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    vocab_int.append(ints)

In [220]:
questions_int = []
answers_int = []
for pair in cornell_data:
    ints = []
    for word in pair['request'].split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    questions_int.append(ints)
    ints = []
    for word in pair['reply'].split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    answers_int.append(ints)

In [221]:
print(len(cornell_data), len(vocab_int), len(questions_int), len(answers_int))

167497 167497 167497 167497


In [222]:
len(new_vocab)

8710

In [223]:
sorted_questions = []
sorted_answers = []
sorted_data = []

max_len = 20

for length in range(1, max_len + 1):
    for i in enumerate(questions_int):
        if(len(i[1]) == length):
            sorted_data += [{'request': questions_int[i[0]], 'reply': answers_int[i[0]]}]

In [232]:
questions = []
answers = []
sorted_texts = []
for data in sorted_data:
    question=""
    answer =""
    for idx in data['request']:
        question += int_to_vocab[idx] + ' '
    for idx in data['reply']:
        answer += int_to_vocab[idx] + ' '
    questions.append(question)
    answers.append(answer)
    sorted_texts += [{'request': question,'reply':answer}]

In [239]:
print(questions[60010].split(),answers[60010].split())

['i', 'dont', 'know', 'come', 'on'] ['come', 'on', 'where', '<EOS>']


In [240]:
sorted_texts[60010]

{'request': 'i dont know come on ', 'reply': 'come on where <EOS> '}

In [195]:
query = 'are you serious'
hay = []
for x in query.split():
    hay.append(vocab_to_int[x])
for data in sorted_data:
    if data['request'] == hay:
        question=""
        answer =""
        for idx in data['request']:
            question += " " + int_to_vocab[idx]
        for idx in data['reply']:
            answer += " " + int_to_vocab[idx]
        print(question + " - " + answer)

 are you serious -  yeah he said that <EOS>
 are you serious -  huh imagine that looks like i am <EOS>
 are you serious -  i dont even know whether a defense is possible <EOS>
 are you serious -  yeah i am out <EOS>
 are you serious -  not ever <EOS>
 are you serious -  how long have you been smoking <EOS>
 are you serious -  that is history fletch you owe us nine hundred and eighteen dollars <EOS>
 are you serious -  of course look i have got some missing persons <UNK> evidence of a large <UNK> <EOS>
 are you serious -  forget it it doesnt matter <EOS>
 are you serious -  yeah i think so <EOS>
 are you serious -  have you ever known me not to be <EOS>
 are you serious -  yeah no more i am finished with that <UNK> <EOS>
 are you serious -  give him the <UNK> <EOS>


## -----------------------------------------------------------------------------------------------------
## -----------------------------------------------------------------------------------------------------
## -----------------------------------------------------------------------------------------------------

## Seq2Seq Model using Keras

In [197]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM

import tensorflow as tf

In [243]:
num_encoder_tokens = len(new_vocab)
num_decoder_tokens = len(new_vocab)
max_encoder_seq_len = max(len(txt.split()) for txt in questions)
max_decoder_seq_len = max(len(txt.split()) for txt in answers)
print('Number of unique tokens: ',num_encoder_tokens)
print('Max encoden length: ',max_encoder_seq_len)
print('Max decoder length: ' , max_decoder_seq_len)

Number of unique tokens:  8710
Max encoden length:  20
Max decoder length:  21


In [247]:
encoder_input_data = np.zeros((len(questions), max_encoder_seq_len, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(questions), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(questions), max_decoder_seq_len, num_decoder_tokens), dtype='float32')

In [248]:
for i, (question, answer) in enumerate(zip(questions, answers)):
    for t, word in enumerate(question.split()):
        encoder_input_data[i,t,questions_vocab_to_int[word]] = 1.
    #encoder_input_data[i, t+1:,questions_vocab_to_int[' ']] = 1.
    for t, word in enumerate(answer.split()):
        decoder_input_data[i,t, answers_vocab_to_int[word]] = 1.
        if t>0:
            decoder_target_data[i, t-1, answers_vocab_to_int[word]] = 1.
    #decoder_input_data[i, t+1:,answers_vocab_to_int[' ']] = 1.
    #decoder_input_data[i, t+1:,answers_vocab_to_int[' ']] = 1.

In [249]:
encoder_inputs  = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [250]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(128, return_state=True, return_sequences= True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [251]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=100, validation_split=0.2)
model.save('s2skeras.h5')

Epoch 1/100
 119/2094 [>.............................] - ETA: 27:55 - loss: 2.0414 - accuracy: 0.0472

KeyboardInterrupt: 